In [73]:
import math
import numpy as np
import heapq as hq
import pandas as pd


def huntington_hill(populations_table,num_seats):
    ### populations is a 2d DataFrame  containing the states' names and populations
    ### num_seats is the desired number of seats to assign
    populations = populations_table[1]
    num_states = len(populations)
    if (num_states > num_seats):
        print("More states than seats!")
        return None
    representatives = np.ones(num_states)
    priorities = populations.values / math.sqrt(2)
    last_state = None
    last_priority = None
    second_last_state = None
    second_last_state_priority = None
    for j in range(num_states,num_seats):
        highest = np.argmax(priorities) # index of state with highest priority. It will receive this seat
        if(highest != last_state):
            # If the state getting this seat is not the same as last one, 
            # update the index and priority of second-last state to receive a seat
            second_last_state = last_state
            second_last_state_priority = last_priority
        # Now record the information about this seat
        last_state = highest
        last_priority = priorities[highest] 
        representatives[highest] +=  1    
        # Update this state's priority
        priorities[highest] = populations[highest]/math.sqrt(representatives[highest] * representatives[highest]+1)
        
    return (representatives, # a 1D array of apportioned representatives
            priorities, # 1D array of  priorities of states for next 
            last_state, # the index of the state last assigned a seat
            last_priority, # the priority for the most recently assigned seat.
            second_last_state, # the index of the second-last state assigned a seat
            second_last_state_priority) # the priority of the seat last assigned to second-last state
  
  

In [74]:
example_populations_table = pd.DataFrame(
    [["JAMMU & KASHMIR",10143700],["HIMACHAL PRADESH",6077900],["PUNJAB",24358999],
    ["CHANDIGARH",900635],["UTTARANCHAL",8489349],["HARYANA",21144564],["DELHI",13850507],
    ["RAJASTHAN",56507188],["UTTAR PRADESH",166197921],["BIHAR",82998509],
    ["SIKKIM",540851],["ARUNACHAL PRADESH",1097968],["NAGALAND",1990036],
    ["MANIPUR",2166788],["MIZORAM",888573],["TRIPURA",3199203],
    ["MEGHALAYA",2318822],["ASSAM",26655528],["WEST BENGAL",80176197],
    ["JHARKHAND",26945829],["ORISSA",36804660],["CHHATTISGARH",20833803],
    ["MADHYA PRADESH",60348023],["GUJARAT",50671017],["DAMAN & DIU",158204],
    ["DADRA & NAGAR HAVELI",220490],["MAHARASHTRA",96878627],["ANDHRA PRADESH",76210007],
    ["KARNATAKA",52850562],["GOA",1347668],["LAKSHADWEEP",60650],
    ["KERALA",31841374],["TAMIL NADU",62405679],["PONDICHERRY",974345],
    ["ANDAMAN & NICOBAR ISLANDS",356152]
    ] )

number_of_seats = 545

representatives, priorities, last_state, last_priority, second_last_state,  second_last_state_priority = huntington_hill(example_populations_table, number_of_seats)

In [75]:
absolute_gaps = last_priority - priorities
print(min(absolute_gaps))

79.08356696041301


The __absolute gap__ for a given state is simply the difference between the priority of the last assigned seat and the state's priority.It reflects how much a state's priority would have to _increase_ for the apportionment results to change. It 

The __normalized gap__ for a given state is the number of _people_ who would have to be added to that state for it to gain a seat, assuming all other states' populations remained the same. The code I have does not correctly compute the gap for the last state to receive a seat. The calculation is correct for all other states. (To get the gap right for the last state, I need to use the info about the second-to-last state.)

In [77]:
normalization_factors = np.array(list(map(math.sqrt, reps*(reps + 1))))
normalized_gaps = (last_priority - priorities) * normalization_factors
# Need to fix computation of normalized gap for last state.
min_normalized_gap = min(normalized_gaps)
normalized_runner_up = np.argmin(normalized_gaps)
print("The normalized runner up:", example_populations_table[0][normalized_runner_up], min_normalized_gap)

The normalized runner up: RAJASTHAN 2332.6301014023743


To do: __write an algorithm to compute the distance to instability. __

2332.6301014023743